In [1]:
dd_image_path = "/home/etinoxa/PycharmProjects/forensic-lab/cfreds_2015_data_leakage_pc.dd"
dd_image_offset = 206848
software_registry_path = "/home/etinoxa/PycharmProjects/forensic-lab/working_files/registry/SOFTWARE"
system_registry_path = "/home/etinoxa/PycharmProjects/forensic-lab/working_files/registry/SYSTEM"

In [3]:
import hashlib
import sys
# Calculate MD% and SHA 256 hash the imgae


def calculate_md5(file_path):
    """
    Calculate the MD5 hash of a file.

    :param file_path: Path to the file.
    :return: MD5 hash as a hexadecimal string.
    """
    md5_hash = hashlib.md5()
    with open(file_path, "rb") as file:
        while True:
            data = file.read(4096)  # Read the file in chunks of 4KB
            if not data:
                break
            md5_hash.update(data)
    return md5_hash.hexdigest()


def calculate_sha256(file_path):
    """
    Calculate the SHA-256 hash of a file.

    :param file_path: Path to the file.
    :return: SHA-256 hash as a hexadecimal string.
    """
    sha256_hash = hashlib.sha256()
    with open(file_path, "rb") as file:
        while True:
            data = file.read(4096)  # Read the file in chunks of 4KB
            if not data:
                break
            sha256_hash.update(data)
    return sha256_hash.hexdigest()


if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("Usage: python 1_calculate_md5_sha.py <file_path>")
        sys.exit(1)

    file_path = dd_image_path

    try:
        md5_hash = calculate_md5(file_path)
        sha256_hash = calculate_sha256(file_path)

        print(f"MD5 Hash: {md5_hash}")
        print(f"SHA-256 Hash: {sha256_hash}")
    except FileNotFoundError:
        print("File not found.")


MD5 Hash: c256cf70f249e45b6152c4fa5a6ff123
SHA-256 Hash: 4dfc34e7ed89aaf53c7ac131fdb44052b2116a970e13c0e0e7ff5a20f14459cc


In [34]:
file_path = dd_image_path
md5_hash = calculate_md5(file_path)
sha256_hash = calculate_sha256(file_path)
print(f"MD5 Hash: {md5_hash}")
print(f"SHA-256 Hash: {sha256_hash}")

MD5 Hash: c256cf70f249e45b6152c4fa5a6ff123
SHA-256 Hash: 4dfc34e7ed89aaf53c7ac131fdb44052b2116a970e13c0e0e7ff5a20f14459cc


In [6]:
import sys
import pytsk3

# 
# Get the name of the disk image from command line argument
# image_filename = sys.argv[1]


image_filename = dd_image_path

# Create an Img_Info object to represent the image file
disk_image = pytsk3.Img_Info(image_filename)

# Create a Volume_Info object to access the partitions
volume_info = pytsk3.Volume_Info(disk_image)

# Loop through each partition and print its attributes
for partition in volume_info:
    print("Partition:", partition.addr)
    print("Description:", partition.desc)
    print("Start sector:", partition.start)
    print("Length:", partition.len)
    print()

Partition: 0
Description: b'Primary Table (#0)'
Start sector: 0
Length: 1

Partition: 1
Description: b'Unallocated'
Start sector: 0
Length: 2048

Partition: 2
Description: b'NTFS / exFAT (0x07)'
Start sector: 2048
Length: 204800

Partition: 3
Description: b'NTFS / exFAT (0x07)'
Start sector: 206848
Length: 41734144

Partition: 4
Description: b'Unallocated'
Start sector: 41940992
Length: 2048


In [ ]:
import argparse
import pytsk3

# Get Sectors of all partitions in the disk image

def list_unallocated_space(image_path):
    # Open the disk image
    img_info = pytsk3.Img_Info(image_path)

    # Open the volume system
    vol_info = pytsk3.Volume_Info(img_info)

    # Print partition table header
    print("Slot      Start        End          Length       Description")

    # Iterate through the partitions
    for part in vol_info:
        start = part.start
        end = start + part.len - 1
        length = part.len
        description = part.desc

        print(f"{part.addr:4}  {start:12}  {end:12}  {length:12}  {description}")


if __name__ == "__main__":
    # Set up command-line argument parsing
    parser = argparse.ArgumentParser(description="List unallocated space on a disk.")
    parser.add_argument("image_path", help="Path to the disk image file.")

    # Parse the arguments
    args = parser.parse_args()

    # List unallocated space
    list_unallocated_space(args.image_path)


In [5]:
list_unallocated_space(dd_image_path)

Slot      Start        End          Length       Description
   0             0             0             1  b'Primary Table (#0)'
   1             0          2047          2048  b'Unallocated'
   2          2048        206847        204800  b'NTFS / exFAT (0x07)'
   3        206848      41940991      41734144  b'NTFS / exFAT (0x07)'
   4      41940992      41943039          2048  b'Unallocated'


In [11]:
import struct
import sys

# Execution: python3 partition_info.py ../cfreds_2015_data_leakage_pc.dd 206848

def read_sector(image_file, offset, sector_size):
    image_file.seek(offset * sector_size)
    return image_file.read(sector_size)

def parse_boot_sector(sector_data):
    # Extract file system information
    file_system_type = sector_data[3:11].decode("utf-8", errors="ignore")
    oem_name = sector_data[71:82].decode("utf-8", errors="ignore")
    version = sector_data[82:90].decode("utf-8", errors="ignore")

    # Extract metadata information
    first_cluster_of_mft, first_cluster_of_mft_mirror = struct.unpack("<QQ", sector_data[48:64])
    size_of_mft_entries, size_of_index_records = struct.unpack("<II", sector_data[64:72])
    mft_range = "0 - " + str(first_cluster_of_mft * size_of_mft_entries)
    root_directory = struct.unpack("<I", sector_data[96:100])[0]

    # Extract content information
    sector_size, cluster_size = struct.unpack("<HH", sector_data[11:15])
    total_cluster_range = "0 - " + str(int(((root_directory * cluster_size) / sector_size) - 1))
    total_sector_range = "0 - " + str(int((root_directory * cluster_size) - 1))

    return {
        "\nFILE SYSTEM INFORMATION": {
            "file_system_type": file_system_type,
            "oem_name": oem_name,
            "version": version,
        },
        "\nMETADATA INFORMATION": {
            "first_cluster_of_mft": first_cluster_of_mft,
            "first_cluster_of_mft_mirror": first_cluster_of_mft_mirror,
            "size_of_mft_entries": size_of_mft_entries,
            "size_of_index_records": size_of_index_records,
            "mft_range": mft_range,
            "root_directory": root_directory,
        },
        "\nCONTENT INFORMATION": {
            "sector_size": sector_size,
            "cluster_size": cluster_size,
            "total_cluster_range": total_cluster_range,
            "total_sector_range": total_sector_range,
        },
    }

if __name__ == "__main__":
    if len(sys.argv) != 3:
        print("Usage: python partition_details.py <image_file> <offset>")
        sys.exit(1)

    image_file_path = dd_image_path
    offset = dd_image_offset
    sector_size = 512

    try:
        with open(image_file_path, "rb") as image_file:
            sector_data = read_sector(image_file, offset, sector_size)
            partition_details = parse_boot_sector(sector_data)

            # Display partition details
            for section, details in partition_details.items():
                print(section)
                for key, value in details.items():
                    print(f"{key}: {value}")

    except FileNotFoundError:
        print("Image file not found.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")



FILE SYSTEM INFORMATION
file_system_type: NTFS    
oem_name:  Hzʍ  
version:   3м

METADATA INFORMATION
first_cluster_of_mft: 786432
first_cluster_of_mft_mirror: 2
size_of_mft_entries: 246
size_of_index_records: 1
mft_range: 0 - 193462272
root_directory: 1718099487

CONTENT INFORMATION
sector_size: 512
cluster_size: 8
total_cluster_range: 0 - 26845303
total_sector_range: 0 - 13744795895


In [40]:
import argparse
import pytsk3

# List all files in partition

def list_files_in_partition(image_path, offset):
    try:
        # Open the disk image
        img_info = pytsk3.Img_Info(image_path)

        # Open the file system at the specified partition offset
        fs_info = pytsk3.FS_Info(img_info, offset=offset * 512)

        # Get the root directory
        root_directory = fs_info.open_dir(path="/")

        # Initialize a list to store file names
        partition_files = []

        # Iterate over all files and directories
        for fs_object in root_directory:
            if fs_object.info.name:
                # Check if the file is unallocated (deleted)
                if pytsk3.TSK_FS_META_FLAG_ALLOC:
                    partition_files.append(fs_object.info.name.name.decode('utf-8'))

        # Print the list of files
        if partition_files:
            print("Partition files:")
            for file_name in partition_files:
                print(file_name)
        else:
            print("No found.")
    except Exception as e:
        print("An error occurred:", str(e))

if __name__ == "__main__":
    # List deleted files
    list_files_in_partition(dd_image_path, dd_image_offset)


Partition files:
Program Files (x86)
Users
$AttrDef
$BadClus
$Bitmap
$Boot
$Extend
$LogFile
$MFT
$MFTMirr
$Recycle.Bin
$Secure
$UpCase
$Volume
.
Config.Msi
Documents and Settings
hiberfil.sys
MSOCache
pagefile.sys
PerfLogs
Program Files
WIMECB2.tmp
WIMECC3.tmp
WIMECF3.tmp
WIMECF4.tmp
WIMED24.tmp
WIMED63.tmp
WIMF282.tmp
WIMF283.tmp
WIMF2A4.tmp
WIMF2A5.tmp
WIMF2A6.tmp
Windows
WinPEpge.sys
ProgramData
Recovery
System Volume Information
WIMF32B.tmp
WIMF33C.tmp
WIMF5BD.tmp
WIMF5BE.tmp
WIMF5BF.tmp
WIMF5C0.tmp
WIMF5C1.tmp
WIMF5D1.tmp
WIMF5D2.tmp
WIMF5D3.tmp
WIMF5D4.tmp
WIMF5D5.tmp
WIMF5D6.tmp
WIMF5D7.tmp
WIMF913.tmp
WIMF914.tmp
WIMFA8B.tmp
WIMFA8C.tmp
WinPEpge.sys
Config.Msi
$OrphanFiles


In [2]:
import argparse
from regipy import RegistryHive

# Get the Windows version information

# Replace this path with the actual path to your SOFTWARE hive file
# hive_path = '../working_files/registry/SOFTWARE'
def get_windows_version(hive_path):
    key_path = r'\Microsoft\Windows NT\CurrentVersion'

    # Load the hive
    registry_hive = RegistryHive(hive_path)

    # registry_hive.get_key()
    key = registry_hive.get_key(key_path)

    # Get the values
    product_name = key.get_value('ProductName')
    current_build = key.get_value('CurrentBuild')
    current_version = key.get_value('CurrentVersion')
    build_lab_ex = key.get_value('BuildLabEx')

    # Print the Windows version information
    print(f'Product Name: {product_name}')
    print(f'Current Build: {current_build}')
    print(f'Current Version: {current_version}')
    print(f'Build Lab Ex: {build_lab_ex}')


if __name__ == "__main__":
    get_windows_version(software_registry_path)

    



Product Name: Windows 7 Ultimate
Current Build: 7601
Current Version: 6.1
Build Lab Ex: 7601.18247.amd64fre.win7sp1_gdr.130828-1532


In [8]:
import argparse
from regipy import RegistryHive
from regipy.exceptions import RegistryKeyNotFoundException
from datetime import datetime

# Get date and time of last shutdown


# Replace this path with the actual path to your SOFTWARE hive file
# hive_path = '../working_files/registry/SYSTEM'

def get_time_zone(hive_path):
    # Load the hive
    registry_hive = RegistryHive(hive_path)

    try:
        timezone_key = registry_hive.get_key('\ControlSet001\Control\TimeZoneInformation')
        day_light_name = timezone_key.get_value('DaylightName')
        standard_name = timezone_key.get_value('StandardName')
        bias = timezone_key.get_value('Bias')
        active_time_bias = timezone_key.get_value('ActiveTimeBias')
        time_zone_key_name = timezone_key.get_value('TimeZoneKeyName').decode('Windows-1252', errors='replace')

        print('DaylightName:', day_light_name)
        print('StandardName:', standard_name)
        print('Bias:', bias, f'({round(bias/60)} hours)')
        print('ActiveTimeBias:', active_time_bias, f'({round(active_time_bias/60)} hours)')
        print('TimeZoneKeyName:', time_zone_key_name)
    except RegistryKeyNotFoundException:
        print('Timezone key not found')

if __name__ == "__main__":
    get_time_zone(system_registry_path)


DaylightName: @tzres.dll,-111
StandardName: @tzres.dll,-112
Bias: 300 (5 hours)
ActiveTimeBias: 240 (4 hours)
TimeZoneKeyName: E a s t e r n   S t a n d a r d   T i m e   a r d   T i m e             ¸Õ“    H               Ü“    èN5     p       ÿ#lýÿÿÿÿ±ÊÿÿÿÿÿC›Øv    ï›Øv                    à>Îúþ  À      ü;Øv    À      à>Îúþ  K                                          þ  


In [9]:
import argparse
from regipy import RegistryHive
from regipy.exceptions import RegistryKeyNotFoundException
from datetime import datetime

# Get User Name

# Replace this path with the actual path to your SYSTEM hive file
# hive_path = '../working_files/registry/SYSTEM'

def get_computer_name(hive_path):
    # Load the hive
    registry_hive = RegistryHive(hive_path)

    try:
        computer_key = registry_hive.get_key('\ControlSet001\Control\ComputerName\ComputerName')
        computer_name = computer_key.get_value('ComputerName')
        print('Computer Name:', computer_name)
    except RegistryKeyNotFoundException:
        print('Computer name key not found')

    try:
        network_key = registry_hive.get_key('\ControlSet001\\Services\\Tcpip\\Parameters')
        hostname = network_key.get_value('Hostname')
        print('Hostname:', hostname)
    except RegistryKeyNotFoundException:
        print('Hostname key not found')

if __name__ == "__main__":
    get_computer_name(system_registry_path)


Computer Name: INFORMANT-PC
Hostname: informant-PC


In [8]:
from regipy import RegistryHive
import argparse

# Get Installed Apps

# hive_path = "../working_files/registry/SOFTWARE"
# Execution = python .\scripts\get_installed_apps.py .\working_files\registry\SOFTWARE


def get_installed_apps(hive_path):
    registry = RegistryHive(hive_path)
    installer_path = r"\Microsoft\Windows\CurrentVersion\Installer\UserData"

    # Iterate over subpath
    interfaces = registry.get_key(installer_path)
    for i in interfaces.iter_subkeys():
        print(i.name)
        products = i.get_subkey("Products")
        for j in products.iter_subkeys():
            properties = j.get_subkey("InstallProperties")
            print(f'{properties.get_value("DisplayName")} {properties.get_value("DisplayVersion")} ({properties.get_value("Publisher")})')


if __name__ == "__main__":
    get_installed_apps(software_registry_path)

S-1-5-18
Microsoft DCF MUI (English) 2013 15.0.4420.1017 (Microsoft Corporation)
Microsoft OneNote MUI (English) 2013 15.0.4420.1017 (Microsoft Corporation)
Microsoft Office 32-bit Components 2013 15.0.4420.1017 (Microsoft Corporation)
Microsoft Office Shared 32-bit MUI (English) 2013 15.0.4420.1017 (Microsoft Corporation)
Microsoft Office OSM MUI (English) 2013 15.0.4420.1017 (Microsoft Corporation)
Microsoft Office OSM UX MUI (English) 2013 15.0.4420.1017 (Microsoft Corporation)
Microsoft InfoPath MUI (English) 2013 15.0.4420.1017 (Microsoft Corporation)
Microsoft Access MUI (English) 2013 15.0.4420.1017 (Microsoft Corporation)
Microsoft Office Shared Setup Metadata MUI (English) 2013 15.0.4420.1017 (Microsoft Corporation)
Microsoft Excel MUI (English) 2013 15.0.4420.1017 (Microsoft Corporation)
Microsoft Access Setup Metadata MUI (English) 2013 15.0.4420.1017 (Microsoft Corporation)
Microsoft PowerPoint MUI (English) 2013 15.0.4420.1017 (Microsoft Corporation)
Microsoft Publisher MU

In [15]:
from regipy import RegistryHive
import argparse

# Get Last Logged On User

# hive_path = "../working_files/registry/SOFTWARE"
# Execution = python .\scripts\get_lastloggedon_user.py .\working_files\registry\SOFTWARE
def get_last_logged_on_user(hive_path):
    key_path = r"\Microsoft\Windows\CurrentVersion\Authentication\LogonUI"

    # Load the hive
    registry_hive = RegistryHive(hive_path)

    key = registry_hive.get_key(key_path)
    try:
        last_logged_on_user = key.get_value('LastLoggedOnUser')
        last_logged_on_sam_user = key.get_value('LastLoggedOnSAMUser')

        print(f"LastLoggedOnUser = {last_logged_on_user}")
        print(f"LastLoggedOnSAMUser = {last_logged_on_sam_user}")

    except FileNotFoundError:
        print("Key or value not found in the registry")
    except PermissionError:
        print("Permission denied. Please run the script with elevated privileges.")



if __name__ == "__main__":
    get_last_logged_on_user(software_registry_path)


LastLoggedOnUser = .\informant
LastLoggedOnSAMUser = informant-PC\informant


In [16]:
from regipy import RegistryHive
import argparse

# Get Network Interface Info

#hive_path = "../working_files/registry/SYSTEM"
# Execution = python .\scripts\get_network_interface_info.py .\working_files\registry\SYSTEM

def get_network_interface_info(hive_path):
    registry = RegistryHive(hive_path)
    tcpip_interfaces_key = r"\ControlSet001\Services\Tcpip\Parameters\Interfaces"

    interfaces = registry.get_key(tcpip_interfaces_key)

    for i in interfaces.iter_subkeys():
        print(f"Adapter: {i.name}")

        for j in i.iter_values(as_json=True):
            print(f"\t{j.name}: {j.value}")
        print("\n")


if __name__ == "__main__":
    get_network_interface_info(system_registry_path)

Adapter: {846ee342-7039-11de-9d20-806e6f6e6963}


Adapter: {E2B9AEEC-B1F7-4778-A049-50D7F2DAB2DE}
	UseZeroBroadcast: 0
	EnableDeadGWDetect: 1
	EnableDHCP: 1
	NameServer: 0
	Domain: 0
	RegistrationEnabled: 1
	RegisterAdapterName: 0
	DhcpIPAddress: 10.11.11.129
	DhcpSubnetMask: 255.255.255.0
	DhcpServer: 10.11.11.254
	Lease: 1800
	LeaseObtainedTime: 1427296790
	T1: 1427297690
	T2: 1427298365
	LeaseTerminatesTime: 1427298590
	AddressType: 0
	IsServerNapAware: 0
	DhcpConnForceBroadcastFlag: 0
	DhcpInterfaceOptions: 2c0000000000000004000000000000001ed912550a0b0b02060000000000000004000000000000001ed912550a0b0b02030000000000000004000000000000001ed912550a0b0b020f000000000000000b000000000000001ed912556c6f63616c646f6d61696e00010000000000000004000000000000001ed91255ffffff00
	DhcpGatewayHardware: 0a0b0b0206000000005056ebb22c
	DhcpGatewayHardwareCount: 1
	DhcpNameServer: 10.11.11.2
	DhcpDefaultGateway: ['10.11.11.2']
	DhcpDomain: localdomain
	DhcpSubnetMaskOpt: ['255.255.255.0']


In [44]:
from regipy import RegistryHive
import datetime
import argparse

# Get last shutdown time

# hive_path = "../working_files/registry/SYSTEM"
# Execution = python .\scripts\get_shutdown_time.py .\working_files\registry\SYSTEM
def get_shutdown_time(hive_path):
    key_path = r"\ControlSet001\Control\Windows"
    # hive_path = "../working_files/registry/SYSTEM"
    # Load the hive
    registry_hive = RegistryHive(hive_path)

    key = registry_hive.get_key(key_path)
    try:
        shutdowntime = key.get_value('ShutdownTime')

        # Convert bytes to an integer
        filetime_int = int.from_bytes(shutdowntime, byteorder='little')

        # Define the FILETIME epoch
        filetime_epoch = datetime.datetime(1601, 1, 1)

        # Convert the integer to a datetime object
        shutdowntime = filetime_epoch + datetime.timedelta(microseconds=filetime_int / 10)

        print(f"ShutdownTime: {shutdowntime}")

    except FileNotFoundError:
        return "Key or value not found in the registry"
    except PermissionError:
        return "Permission denied. Please run the script with elevated privileges."


if __name__ == "__main__":
    # Set up command-line argument parsing
    # parser = argparse.ArgumentParser(description="List last recorded shutdown date/time.")
    # parser.add_argument("hive_path", help="Path to the registry file.")

    # Parse the arguments
    # args = parser.parse_args()

    # Print Windows version
    get_shutdown_time(system_registry_path)

ShutdownTime: 2015-03-25 15:31:05.232008
